# IMPORTANDO BIBLIOTECAS

In [22]:
# Importando bibliotecas necessárias
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# EXIBIÇÃO DOS DATAFRAMES

In [2]:
# Configurando a exibição do dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

# LEITURA DOS ARQUIVOS

In [50]:
# Lista com caminhos locais distintos
diretorios_locais = [
    r'C:\Users\brand\OneDrive\Área de Trabalho\Fraud_Detection_ML\Data\\',
    r'C:\Users\Giovanni Maia\Desktop\Dados\\'
]

diretorio_github= 'https://media.githubusercontent.com/media/jeanbrandao01/Fraud_Detection_ML/dev/Data/'

def carregar_dataframes(diretorios_locais, diretorio_github):
    for diretorio_local in diretorios_locais:
        try:
            # Tente carregar o DataFrame de fraudTest.csv a partir do diretório local
            df_test = pd.read_csv(diretorio_local + 'fraudTest.csv')

            # Tente carregar o DataFrame de fraudTrain.csv a partir do diretório local
            df_train = pd.read_csv(diretorio_local + 'fraudTrain.csv')

            # Se o carregamento for bem-sucedido, retorne os DataFrames
            return df_test, df_train
        except FileNotFoundError:
            continue  # Continue para o próximo diretório local em caso de erro

    # Se nenhum diretório local contiver os arquivos, carregue do GitHub
    url_test = diretorio_github + 'fraudTest.csv'
    url_train = diretorio_github + 'fraudTrain.csv'

    df_test = pd.read_csv(url_test)
    df_train = pd.read_csv(url_train)

    return df_test, df_train

df_test, df_train = carregar_dataframes(diretorios_locais,diretorio_github)


# EXIBIÇÃO DOS DATAFRAMES ORIGINAIS

In [24]:
# Dataframe "fraudTest.csv"
df_test.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,11710,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,32780,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,49632,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [25]:
# Dataframe "fraudTrain.csv"
df_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


# ANÁLISE DESCRITIVA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [26]:
# Algumas estatisticas do dataset
df_train.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [7]:
# Verificando as colunas existentes
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [ ]:
# Realizando a contagem de valores da coluna "merchant"
df_train.merchant.value_counts()

In [ ]:
# Realizando a contagem de valores da coluna "is_fraud"
print("Contagem de valores da coluna is_fraud:")
print("-"*39)
print(df_train.is_fraud.value_counts())


In [ ]:
# Verificando a extensão do dataset
print("Extensão do dataset:")
print("-"*23)
print(df_train.shape)


In [ ]:
# Verificando os nomes de cada coluna 
print("Colunas do dataset")
print(df_train.columns)

# ANÁLISE EXPLORATÓRIA: DATASET DE TREINO - FRAUDTRAIN.CSV

In [ ]:
# Calcular a proporção de fraudes
fraud_proportion = df_train['is_fraud'].value_counts(normalize=True)

# Criar um DataFrame para facilitar a criação do gráfico
fraud_data = pd.DataFrame({'Classe de Fraude': fraud_proportion.index, 'Proporção': fraud_proportion.values})

# Criar o gráfico de barras com o Plotly
fig = px.bar(fraud_data, x='Classe de Fraude', y='Proporção', text='Proporção', title='Distribuição de Fraudes', labels={'Classe de Fraude': 'Classe de Fraude'})
fig.update_traces(texttemplate='%{text:.2%}', textposition='outside')
fig.show()



In [ ]:
# mostrando a porcentagem de transações fraudulentas e não fraudulentas para cada categoria presente na coluna category 
round(pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index')*100,2)

In [8]:
# Cria uma tabela de contingência
crosstab = pd.crosstab(index=df_train.category, columns=df_train.is_fraud, normalize='index') * 100

# Reinicia o índice para facilitar o manuseio
crosstab = crosstab.reset_index()

# Cria um gráfico de barras empilhadas usando o Plotly Express
fig = px.bar(crosstab, x='category', y=[0, 1], title="Gráfico de Barras Empilhadas da Tabela de Contingência (Percentual)",
             labels={0: "Não Fraude", 1: "Fraude"},
             height=400)

# Atualiza o layout para um gráfico de barras empilhadas
fig.update_layout(barmode='stack')

# Mostra o gráfico
fig.show()


In [ ]:
# Agrupa os dados por estado e calcula o número total de transações em cada estado
total_transacoes_por_estado = df_train["state"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por estado e calcula o número de transações fraudulentas em cada estado
transacoes_fraudulentas_por_estado = dados_fraudulentos["state"].value_counts()

# Calcula a taxa de fraude para cada estado (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_estado / total_transacoes_por_estado).fillna(0)

# Reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude.reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Retorna as últimas 20 linhas (estados) dessa Series, representando os estados com o menor número de transações. 
total_transacoes_por_estado.tail(20)

## Fraudes por cidade

In [ ]:
# Retorna o número de cidades únicas presentes na coluna cidade
df_train.city.nunique()

In [ ]:
# Agrupa os dados por cidade e calcula o número total de transações em cada cidade
total_transacoes_por_cidade = df_train["city"].value_counts()

# Filtra os dados para transações fraudulentas
dados_fraudulentos = df_train[df_train["is_fraud"] == 1]

# Agrupa os dados fraudulentos por cidade e calcula o número de transações fraudulentas em cada cidade
transacoes_fraudulentas_por_cidade = dados_fraudulentos["city"].value_counts()

# Calcula a taxa de fraude para cada cidade (transações fraudulentas / total de transações)
taxa_de_fraude = (transacoes_fraudulentas_por_cidade / total_transacoes_por_cidade).fillna(0)

# Seleciona as cidades com taxa de fraude igual a 1, reinicia o índice e ordena os resultados por 'count' de forma descendente
taxa_de_fraude[taxa_de_fraude == 1].reset_index().sort_values(by='count', ascending=False)


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um violin plot usando o Seaborn para valor da compra por classe
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.violinplot(data=df_train.query('amt <= 200'), x='is_fraud', y='amt')
# Define o título e rótulos
plt.title('Violin Plot - Distribuição do Valor da Compra por Classe de Fraude')
plt.xlabel('Classe de Fraude')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um boxplot usando o Seaborn para distruibuição do valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.boxplot(data=df_train.query('amt <= 200'), y='amt', color='skyblue')  # Adicionado cor para melhor visualização

# Define o título e rótulos
plt.title('Boxplot - Distribuição do Valor da Compra')
plt.ylabel('Valor da Compra')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Escolhe uma paleta de cores para as barras (cores mais claras)
bar_palette = sns.color_palette("Blues", as_cmap=True)

# Escolhe uma cor mais escura para a linha do KDE
line_color = 'navy'

# Cria um histograma com KDE usando o Seaborn para valor de compra
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt <= 200'), x='amt', kde=True)
sns.kdeplot(data=df_train.query('amt <= 200')['amt'], color=line_color, linewidth=2)

# Define o título e rótulos
plt.title('Histograma com KDE - Valor da Compra')
plt.xlabel('Valor da Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Define o estilo (opcional)
sns.set_style("whitegrid")

# Cria um histograma com KDE usando o Seaborn para valores de compra acima de 2000
plt.figure(figsize=(10, 6))  # Define o tamanho da figura (opcional)
sns.histplot(data=df_train.query('amt > 2000'), x='amt', kde=True, color='red')  # Adiciona cor para melhor visualização

# Define o título e rótulos
plt.title('Histograma com KDE - Valor de Compra Acima de 2000')
plt.xlabel('Valor de Compra')
plt.ylabel('Contagem')

# Mostra o gráfico
plt.show()


In [ ]:
# Selecionar apenas as colunas numéricas
numeric_columns = df_train.select_dtypes(include=['int64', 'float64'])

# Calculando a matriz de correlação para colunas numéricas
correlation_matrix = numeric_columns.corr()

# Criando uma figura maior
plt.figure(figsize=(20, 10))

# Criando um mapa de calor de correlação com paleta de cores personalizada
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', center=0, cbar_kws={'label': 'Correlação'})
plt.title('Mapa de Calor de Correlação')

plt.show()


In [ ]:
# Análise temporal

# Convertendo a coluna 'trans_date_trans_time' para um objeto de data e hora
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])

# Extraindo características de data e hora
df_train['ano'] = df_train['trans_date_trans_time'].dt.year
df_train['mes'] = df_train['trans_date_trans_time'].dt.month
df_train['dia_da_semana'] = df_train['trans_date_trans_time'].dt.day_name()
df_train['hora'] = df_train['trans_date_trans_time'].dt.hour
df_train['minuto'] = df_train['trans_date_trans_time'].dt.minute
df_train['segundo'] = df_train['trans_date_trans_time'].dt.second

# Filtrando fraudes
frauds = df_train[df_train['is_fraud'] == 1]

# Contando fraudes por mês
frauds_by_month = frauds['mes'].value_counts().sort_index().reset_index()
frauds_by_month.columns = ['Mês', 'Número de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(frauds_by_month, x='Mês', y='Número de Fraudes', title='Fraudes por Mês',
             labels={'Mês': 'Mês', 'Número de Fraudes': 'Número de Fraudes'})

# Adicionando números em cima das barras
fig.update_traces(text=frauds_by_month['Número de Fraudes'], textposition='outside')

fig.show()


In [ ]:
# Contagem de ocorrências por categoria
category_counts = df_train['category'].value_counts().reset_index()
category_counts.columns = ['Categoria', 'Contagem']

# Agrupando por categoria e calculando a proporção de fraudes
fraud_proportion_by_category = df_train.groupby('category')['is_fraud'].mean().reset_index()
fraud_proportion_by_category.columns = ['Categoria', 'Proporção de Fraudes']

# Criando um gráfico de barras interativo com Plotly
fig = px.bar(fraud_proportion_by_category, x='Categoria', y='Proporção de Fraudes', title='Proporção de Fraudes por Categoria',
             labels={'Categoria': 'Categoria', 'Proporção de Fraudes': 'Proporção de Fraudes'})

# Formatando os números com cinco casas decimais
fraud_proportion_by_category['Proporção de Fraudes'] = fraud_proportion_by_category['Proporção de Fraudes'].round(5)

# Adicionando os números formatados (com cinco casas decimais) em cima das barras
fig.update_traces(text=fraud_proportion_by_category['Proporção de Fraudes'], textposition='outside')

fig.show()


# FEATURE ENGINEERING

In [16]:
df_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [51]:
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])


df_train.sort_values(by=['cc_num' ,'trans_date_trans_time'], ascending=True, inplace=True)


agrupado_cc_valor = df_train.groupby('cc_num')['amt']

media_cartao = agrupado_cc_valor.transform('mean')
desv_pad_cartao = agrupado_cc_valor.transform('std')

df_train['media'] = media_cartao

df_train['amt_score_z'] = (df_train['amt'] - media_cartao) / desv_pad_cartao

df_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,media,amt_score_z
1017,1017,2019-01-01 12:47:15,60416207185,"fraud_Jones, Sawayn and Romaguera",misc_net,7.27,Mary,Diaz,F,9886 Anita Drive,Fort Washakie,WY,82514,43.0048,-108.8964,1645,Information systems manager,1986-02-17,98e3dcf98101146a577f85a34e58feec,1325422035,43.974711,-109.741904,0,56.023366,-0.397556
2724,2724,2019-01-02 08:44:57,60416207185,fraud_Berge LLC,gas_transport,52.94,Mary,Diaz,F,9886 Anita Drive,Fort Washakie,WY,82514,43.0048,-108.8964,1645,Information systems manager,1986-02-17,498120fc45d277f7c88e3dba79c33865,1325493897,42.018766,-109.044172,0,56.023366,-0.025143
2726,2726,2019-01-02 08:47:36,60416207185,fraud_Luettgen PLC,gas_transport,82.08,Mary,Diaz,F,9886 Anita Drive,Fort Washakie,WY,82514,43.0048,-108.8964,1645,Information systems manager,1986-02-17,95f514bb993151347c7acdf8505c3d62,1325494056,42.961335,-109.157564,0,56.023366,0.212477
2882,2882,2019-01-02 12:38:14,60416207185,fraud_Daugherty LLC,kids_pets,34.79,Mary,Diaz,F,9886 Anita Drive,Fort Washakie,WY,82514,43.0048,-108.8964,1645,Information systems manager,1986-02-17,4f0c1a14e0aa7eb56a490780ef9268c5,1325507894,42.228227,-108.747683,0,56.023366,-0.173146
2907,2907,2019-01-02 13:10:46,60416207185,fraud_Beier and Sons,home,27.18,Mary,Diaz,F,9886 Anita Drive,Fort Washakie,WY,82514,43.0048,-108.8964,1645,Information systems manager,1986-02-17,3b2ebd3af508afba959640893e1e82bc,1325509846,43.321745,-108.091143,0,56.023366,-0.235201


In [57]:
df_train.loc[df_train.is_fraud == 0, 'amt_score_z'].mean()

-0.018655857471345974

In [61]:
df_train.select_dtypes(exclude='object').corr(method='pearson')

,Unnamed: 0,trans_date_trans_time,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,media,amt_score_z
Unnamed: 0,1.000000,0.998966,3.857663e-04,-0.000251,7.093943e-04,6.016648e-04,-6.762618e-04,-1.678021e-03,0.998971,0.000541,-0.000671,-0.004767,-6.735765e-04,-7.005652e-04
trans_date_trans_time,0.998966,1.000000,3.537740e-04,-0.000285,6.669846e-04,6.358529e-04,-6.382351e-04,-1.715644e-03,0.999996,0.000564,-0.000632,-0.005041,-7.667477e-04,-7.605403e-04
cc_num,0.000386,0.000354,1.000000e+00,0.001769,4.145895e-02,-5.927062e-02,-4.827780e-02,-8.991067e-03,0.000354,-0.058942,-0.048252,-0.000981,1.461406e-02,-3.803664e-18
amt,-0.000251,-0.000285,1.769400e-03,1.000000,1.843402e-03,-1.925520e-03,-1.874756e-04,5.818284e-03,-0.000293,-0.001873,-0.000151,0.219404,1.210752e-01,8.812500e-01
zip,0.000709,0.000667,4.145895e-02,0.001843,1.000000e+00,-1.142904e-01,-9.097321e-01,7.846706e-02,0.000670,-0.113561,-0.908924,-0.002162,1.522527e-02,4.899489e-18
lat,0.000602,0.000636,-5.927062e-02,-0.001926,-1.142904e-01,1.000000e+00,-1.553342e-02,-1.557296e-01,0.000632,0.993592,-0.015509,0.001894,-1.590351e-02,8.189004e-19
long,-0.000676,-0.000638,-4.827780e-02,-0.000187,-9.097321e-01,-1.553342e-02,1.000000e+00,-5.271464e-02,-0.000642,-0.015452,0.999120,0.001721,-1.548423e-03,-1.347567e-17
city_pop,-0.001678,-0.001716,-8.991067e-03,0.005818,7.846706e-02,-1.557296e-01,-5.271464e-02,1.000000e+00,-0.001714,-0.154781,-0.052687,0.002136,4.805514e-02,3.078883e-18
unix_time,0.998971,0.999996,3.536641e-04,-0.000293,6.704698e-04,6.320844e-04,-6.417884e-04,-1.714003e-03,1.000000,0.000561,-0.000635,-0.005078,-7.851340e-04,-7.704697e-04
merch_lat,0.000541,0.000564,-5.894213e-02,-0.001873,-1.135614e-01,9.935924e-01,-1.545181e-02,-1.547814e-01,0.000561,1.000000,-0.015431,0.001741,-1.585275e-02,2.607635e-05


In [ ]:
# Preparação dos dados e divisão em treino/teste
features = df_train[['amt', 'category', 'gender', 'city', 'state', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'dia_da_semana', 'hora', 'minuto', 'segundo']]
labels = df_train['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)


In [ ]:
# Definição de transformadores para Pré-processamento
numericas_continuas = ['amt', 'zip', 'lat', 'long', 'unix_time', 'merch_lat', 'merch_long', 'ano', 'mes', 'hora', 'minuto', 'segundo']
string_categoricas = ['category', 'gender', 'city', 'state', 'dia_da_semana']

# PIPELINE

In [ ]:

# Transformadores para variáveis numéricas contínuas
numeric_transformer = StandardScaler()

# Transformadores para variáveis categóricas
categorical_transformer = OneHotEncoder()

# Lista de tuplas indicando os transformadores a serem aplicados a cada conjunto de features
transformers = [
    ('num_continuas', numeric_transformer, numericas_continuas),
    ('str_categoricas', categorical_transformer, string_categoricas)
]

# Criação do ColumnTransformer integrado ao Pipeline
preprocessor = ColumnTransformer(transformers=transformers)


# TREINAMENTO DO MODELO

In [ ]:
# Treinamento do pré-processador
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


# CLASSIFICADORES

In [ ]:
# Lista de classificadores
classifiers_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
]


In [ ]:
# Função para treinar, avaliar e prever para cada classificador
def train_evaluate_and_predict(classifier, X_train_transformed, y_train, X_test_transformed, y_test):
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_transformed, y_train)
    y_pred = pipe.predict(X_test_transformed)
    
    accuracy = accuracy_score(y_test, y_pred)   
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    result = {
        "Classifier": classifier.__class__.__name__,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Predictions": y_pred
    }

    # Adicionar métricas sem as previsões
    result_without_predictions = result.copy()
    result_without_predictions.pop("Predictions", None)

    return result, result_without_predictions

# Execução em paralelo para todos os classificadores
results, results_without_predictions = zip(*Parallel(n_jobs=-1)(
    delayed(train_evaluate_and_predict)(
        classifier, X_train_transformed, y_train, X_test_transformed, y_test
    ) for classifier in classifiers_list
))

# MÉTRICAS

In [ ]:
# Exiba os resultados sem as previsões
print("\nResults without Predictions:")
for result in results_without_predictions:
    print(result)
    print("---")  # Adiciona uma linha separadora entre os resultados de diferentes classificadores


In [ ]:
# Escolha o classificador com a maior F1-score (média harmônica de precisão e recall)
best_classifier_f1 = max(results_without_predictions, key=lambda x: (2 * x['Precision'] * x['Recall']) / (x['Precision'] + x['Recall']))

# Exiba o resultado
print("Melhor classificador (baseado no F1-score):")
print(best_classifier_f1)


In [ ]:
# Defina pesos para as métricas (pode ajustar conforme necessário)
weights = {
    'Accuracy': 1,
    'Precision': 1,
    'Recall': 1,
    'F1-score': 1
}

# Crie uma lista para armazenar as pontuações agregadas de cada classificador
scores = []

# Calcule as pontuações agregadas para cada classificador
for result in results_without_predictions:
    classifier_name = result['Classifier']
    score = sum(result[metric] * weights[metric] for metric in weights) / sum(weights.values())
    scores.append({'Classifier': classifier_name, 'Score': score})

# Encontre o classificador com a pontuação mais alta
best_classifier = max(scores, key=lambda x: x['Score'])

# Exiba o resultado
print("Melhor classificador (baseado na média ponderada das métricas):")
print(best_classifier)


# CROSS VALIDATION 

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Criar o pipeline com o classificador
classifier_cv = RandomForestClassifier()

# Criar o pipeline com o classificador
pipe_cv = Pipeline(steps=[('classifier', classifier_cv)])

# Definir a estratégia de validação cruzada (Stratified K-Fold)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definir as métricas desejadas
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

# Executar a validação cruzada e calcular as métricas
cv_results = cross_validate(pipe_cv, X_train_transformed, y_train, cv=stratified_kfold, scoring=scoring_metrics)

# Exibir os resultados
for metric in scoring_metrics:
    print(f"{metric.capitalize()} médio na validação cruzada:", cv_results[f'test_{metric}'].mean())
    print(f"Desvio padrão dos {metric.capitalize()} na validação cruzada:", cv_results[f'test_{metric}'].std())
    print("---")


# MATRIZ DE CONFUSÃO PARA TODOS OS INDICADORES (Não usei a cv pra esse conjunto, apenas a validação normal)

In [ ]:
# Lista de todas as colunas de previsão
prediction_columns = [
    'LogisticRegression_Prediction',
    'DecisionTreeClassifier_Prediction',
    'RandomForestClassifier_Prediction',
    'AdaBoostClassifier_Prediction',
    'GradientBoostingClassifier_Prediction'
]

# Loop para criar matrizes de confusão interativas para cada classificador
for classifier_column in prediction_columns:
    # Criar a matriz de confusão
    conf_matrix = confusion_matrix(df_predictions['is_fraud'], df_predictions[classifier_column])

    # Configurações do heatmap
    heatmap = ff.create_annotated_heatmap(
        z=conf_matrix,
        x=['Not Fraud', 'Fraud'],
        y=['Not Fraud', 'Fraud'],
        colorscale='Blues'
    )

    # Atualizar layout para adicionar rótulos
    heatmap.update_layout(
        xaxis=dict(title='Predicted'),
        yaxis=dict(title='True'),
        title=f'Confusion Matrix: is_fraud vs. {classifier_column}'
    )

    # Exibir o gráfico
    heatmap.show()


# DATA FRAME FINAL (AQUI É UM DF QUE TEM OS DADOS NORMAIS E OS DADOS DE PREDIÇÃO DOS 5 CLASSIFICADORES (Nome e predição)) 

In [ ]:
# Criação do DataFrame final
df_predictions = pd.DataFrame(df_train)  # Copiando o DataFrame original

# Adição das previsões ao DataFrame final
for result in results:
    classifier_name = result["Classifier"]
    predictions = result["Predictions"]
    
    # Ajuste para garantir que o número de previsões corresponda ao número de linhas no DataFrame original
    if len(predictions) != len(df_predictions):
        predictions = [0] * (len(df_predictions) - len(predictions)) + list(predictions)

    df_predictions[f"{classifier_name}_Prediction"] = predictions


# SALVANDO O ARQUIVO JÁ TRATADO

In [ ]:
# DataFrame
df_predictions.to_csv(
    'V.Final.csv',  # Caminho e nome do arquivo
    index=False,  # Não incluir índices no arquivo CSV
    encoding='utf-8',  # Codificação do arquivo (pode ser 'utf-8', 'latin1', etc.)
    sep=',',  # Delimitador de colunas (pode ser ',', ';', '\t', etc.)
    header=True  # Incluir cabeçalho no arquivo CSV
)
